In [4]:
# code from https://www.tensorflow.org/lite/guide/signatures

import tensorflow as tf

# Example Model

In [5]:
class Model(tf.Module):

  @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.float32)])
  def encode(self, x):
    result = tf.strings.as_string(x)
    return {
         "encoded_result": result
    }

  @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string)])
  def decode(self, x):
    result = tf.strings.to_number(x)
    return {
         "decoded_result": result
    }

# Convert a model from Signatures

### 1. From Saved Model

In [6]:
model = Model()

# Save the model
SAVED_MODEL_PATH = 'saved_models/coding'

tf.saved_model.save(
    model, SAVED_MODEL_PATH,
    signatures={
      'encode': model.encode.get_concrete_function(),
      'decode': model.decode.get_concrete_function()
    })

# Convert the saved model using TFLiteConverter
converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_PATH)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
]
tflite_model = converter.convert()

# Print the signatures from the converted model
interpreter = tf.lite.Interpreter(model_content=tflite_model)
signatures = interpreter.get_signature_list()
print(signatures)

INFO:tensorflow:Assets written to: saved_models/coding/assets


INFO:tensorflow:Assets written to: saved_models/coding/assets
2022-07-09 12:35:44.822882: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-07-09 12:35:44.822905: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.


Estimated count of arithmetic ops: 0  ops, equivalently 0  MACs
{'decode': {'inputs': ['x'], 'outputs': ['decoded_result']}, 'encode': {'inputs': ['x'], 'outputs': ['encoded_result']}}


2022-07-09 12:35:44.823090: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: saved_models/coding
2022-07-09 12:35:44.823303: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-07-09 12:35:44.823313: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: saved_models/coding
2022-07-09 12:35:44.823746: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-07-09 12:35:44.831677: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: saved_models/coding
2022-07-09 12:35:44.837621: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 14532 microseconds.
2022-07-09 12:35:44.857130: W tensorflow/compiler/mlir/lite/flatbuffer_export.cc:1892] TFLite interpreter needs to link Flex delegate in order to run the model since it contains the following Select TFop(s):
Flex ops: FlexAsString, Flex

### 2. From Keras Model

In [7]:
# Generate a Keras model.
keras_model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(2, input_dim=4, activation='relu', name='x'),
        tf.keras.layers.Dense(1, activation='relu', name='output'),
    ]
)

# Convert the keras model using TFLiteConverter.
# Keras model converter API uses the default signature automatically.
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
tflite_model = converter.convert()

# Print the signatures from the converted model
interpreter = tf.lite.Interpreter(model_content=tflite_model)

signatures = interpreter.get_signature_list()
print(signatures)

INFO:tensorflow:Assets written to: /tmp/tmph8w6ub7v/assets


INFO:tensorflow:Assets written to: /tmp/tmph8w6ub7v/assets
2022-07-09 12:37:51.337908: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-07-09 12:37:51.337928: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-07-09 12:37:51.338119: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmph8w6ub7v


{'serving_default': {'inputs': ['x_input'], 'outputs': ['output']}}


2022-07-09 12:37:51.338706: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-07-09 12:37:51.338716: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmph8w6ub7v
2022-07-09 12:37:51.340299: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-07-09 12:37:51.349780: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmph8w6ub7v
2022-07-09 12:37:51.353960: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 15841 microseconds.


### 3. From Concreate Functions

In [8]:
model = Model()

# Convert the concrete functions using TFLiteConverter
converter = tf.lite.TFLiteConverter.from_concrete_functions(
    [model.encode.get_concrete_function(),
     model.decode.get_concrete_function()], model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
]
tflite_model = converter.convert()

# Print the signatures from the converted model
interpreter = tf.lite.Interpreter(model_content=tflite_model)
signatures = interpreter.get_signature_list()
print(signatures)

INFO:tensorflow:Assets written to: /tmp/tmpio1qddgg/assets


INFO:tensorflow:Assets written to: /tmp/tmpio1qddgg/assets
2022-07-09 12:38:19.129906: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-07-09 12:38:19.129926: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.


Estimated count of arithmetic ops: 0  ops, equivalently 0  MACs
{'decode': {'inputs': ['x'], 'outputs': ['decoded_result']}, 'encode': {'inputs': ['x'], 'outputs': ['encoded_result']}}


2022-07-09 12:38:19.130111: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpio1qddgg
2022-07-09 12:38:19.130346: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-07-09 12:38:19.130355: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpio1qddgg
2022-07-09 12:38:19.130882: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-07-09 12:38:19.139398: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmpio1qddgg
2022-07-09 12:38:19.142520: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 12409 microseconds.
2022-07-09 12:38:19.159427: W tensorflow/compiler/mlir/lite/flatbuffer_export.cc:1892] TFLite interpreter needs to link Flex delegate in order to run the model since it contains the following Select TFop(s):
Flex ops: FlexAsString, FlexStringToN

# Run Signatures

TensorFlow inference APIs support the signature-based executions:   
   
* Accessing the input/output tensors through the names of the inputs and outputs, specified by the signature.
* Running each entry point of the graph separately, identified by the signature key.
* Support for the SavedModel's initialization procedure.

### Python

In [9]:
# Load the TFLite model in TFLite Interpreter
interpreter = tf.lite.Interpreter(model_content=tflite_model)

# Print the signatures from the converted model
signatures = interpreter.get_signature_list()
print('Signature:', signatures)

# encode and decode are callable with input as arguments.
encode = interpreter.get_signature_runner('encode')
decode = interpreter.get_signature_runner('decode')

# 'encoded' and 'decoded' are dictionaries with all outputs from the inference.
input = tf.constant([1, 2, 3], dtype=tf.float32)
print('Input:', input)
encoded = encode(x=input)
print('Encoded result:', encoded)
decoded = decode(x=encoded['encoded_result'])
print('Decoded result:', decoded)

Signature: {'decode': {'inputs': ['x'], 'outputs': ['decoded_result']}, 'encode': {'inputs': ['x'], 'outputs': ['encoded_result']}}
Input: tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32)
Encoded result: {'encoded_result': array([b'1.000000', b'2.000000', b'3.000000'], dtype=object)}
Decoded result: {'decoded_result': array([1., 2., 3.], dtype=float32)}


2022-07-09 12:41:00.880310: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9876 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:26:00.0, compute capability: 8.6


: 